# Deploy a PyTorch Model as a Web Service Azure ML

Note:
* Please use the "Python 3.6 - PyTorch 1.1" kernel for this notebook or install appropriate library versions below.

In [ ]:
from azureml.core.workspace import Workspace
from azureml.core.conda_dependencies import CondaDependencies 

from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice
from azureml.core.model import Model

## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [ ]:
ws = Workspace.from_config(path='config.json')


## Deploy model as web service
Once you have your trained model, you can deploy the model on Azure. In this tutorial, we will deploy the model as a web service in Azure Container Instances (ACI). For more information on deploying models using Azure ML, refer to [Deploy models with Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where).

**Create scoring script**

First, we will create a scoring script that will be invoked by the web service call. Note that the scoring script must have two required functions:

`init()`: 
In this function, you typically load the model into a global object. This function is executed only once when the Docker container is started.


`run(input_data)`: 
In this function, the model is used to predict a value based on the input data. The input and output typically use JSON as serialization and deserialization format, but you are not limited to that.
Refer to the scoring script pytorch_score.py for this tutorial. Our web service will use this file to predict on the new image. When writing your own scoring script, don't forget to test it locally first before you go and deploy the web service.

**Create environment file**

Then, we will need to create an environment file (myenv.yml) that specifies all of the scoring script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image by Azure ML. In this case, we need to specify azureml-core, torch and torchvision.

### Create environment file

In [ ]:
myenv = CondaDependencies.create(pip_packages=['azureml-defaults', 
                                               'torch==1.1.0', 
                                               'torchvision==0.3.0',
                                               'Pillow'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

### Get registered model

In [ ]:
model = Model(ws,'suspicious-behavior-pytorch')#.download()
print(model.name, model.id, model.version, sep='\t')

### Set up inference config

This will be used in the local deployment as well as the deployment to ACI.

In [ ]:
inference_config = InferenceConfig(runtime="python", 
                                   entry_script="pytorch_score.py",
                                   conda_file="myenv.yml")

### Deploy locally to test

Always a good idea.

In [ ]:
# Set up deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=6789)

# Create service
local_service = Model.deploy(ws, 'test', [model], inference_config, deployment_config)
local_service.wait_for_deployment()

In [ ]:
local_service.get_logs()

Send a test image to the local service.

In [ ]:
import numpy as np
input_data = Image.open('Fight_RunAway1frame0016.jpg')
input_data = np.asarray(input_data)
result = local_service.run(input_data=json.dumps({'data': input_data.tolist()}))
print(result)

### Deploy to ACI container
We are ready to deploy. Create an inference configuration which gives specifies the inferencing environment and scripts. Create a deployment configuration file to specify the number of CPUs and gigabytes of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. This cell will run for about 7-8 minutes.

In [ ]:

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=3, 
                                               tags={'data': 'suspicious-behavior',  
                                                     'method':'transfer learning', 
                                                     'framework':'pytorch'},
                                               description='Classify normal/suspicious behavior in PyTorch')

service = Model.deploy(workspace=ws, 
                           name='aci-suspicious-behavior', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

If your deployment fails for any reason and you need to redeploy, make sure to delete the service before you do so: `service.delete()`

**Tip**: If something goes wrong with the deployment, the first thing to look at is the logs from the service by running the following command:

In [ ]:
service.get_logs()

Get the web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
print(service.scoring_uri)

### Test the web service
Finally, let's test our deployed web service. We will send the data as a JSON string to the web service hosted in ACI and use the SDK's run API to invoke the service. Here we will take an image from our validation data to predict on.

In [ ]:
import json
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline
plt.imshow(Image.open('test_images/Fight_RunAway1frame0016.jpg'))

In [ ]:
import numpy as np
input_data = Image.open('test_images/Fight_RunAway1frame0016.jpg')
input_data = np.asarray(input_data)
result = service.run(input_data=json.dumps({'data': input_data.tolist()}))
print(result)

## Clean up
Once you no longer need the web service, you can delete it with a simple API call.

In [ ]:
# service.delete()